In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import torch
import os
import tempfile
import math
import random
from typing import Dict
import nest_asyncio
nest_asyncio.apply()
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from fastapi import Form

from l2cs import Pipeline

# --- 시드 고정 코드 ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # CUDA의 비결정적 연산을 최대한 방지 (성능이 약간 저하될 수 있음)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# --------------------

# ==============================================================================
# 1. FastAPI 애플리케이션 및 AI 모델 초기화 (변경 없음)
# ==============================================================================
print("FastAPI 서버를 시작합니다. AI 모델을 로딩합니다...")
app = FastAPI(title="AI Interview Analysis Server")
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5
)
device = "cuda" if torch.cuda.is_available() else "cpu"
try:
    gaze_pipeline = Pipeline(
        weights='models/L2CSNet_gaze360.pkl', arch='ResNet50', device=torch.device(device)
    )
    print(f"AI 모델 로딩 완료. Device: {device}")
except Exception as e:
    print(f"Error loading L2CS-Net model: {e}")

# ==============================================================================
# 2. FastAPI 엔드포인트 정의 (내부 로직 수정)
# ==============================================================================
@app.post("/calibrate", response_model=Dict[str, float])
async def analyze_video(video_file: UploadFile = File(...)):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video:
            contents = await video_file.read()
            temp_video.write(contents)
            temp_video_path = temp_video.name
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"임시 파일 생성에 실패했습니다: {e}")

    head_yaws, head_pitches, gaze_yaws, gaze_pitches = [], [], [], []
    cap = cv2.VideoCapture(temp_video_path)
    if not cap.isOpened():
        os.unlink(temp_video_path)
        raise HTTPException(status_code=400, detail="업로드된 비디오 파일을 열 수 없거나 손상되었습니다.")

    # ( ★★★ 1. 프레임 카운터 초기화 ★★★ )
    # 분석 결과를 저장할 리스트 초기화
    head_yaws, head_pitches = [], []
    gaze_yaws, gaze_pitches = [], []

    # --- EMA 필터를 위한 변수 초기화 ---
    # 첫 프레임의 값을 저장하기 위해 None으로 초기화합니다.
    smooth_head_yaw, smooth_head_pitch = None, None
    smooth_gaze_yaw, smooth_gaze_pitch = None, None
    alpha = 0.2 # 스무딩 강도 (이 값을 조절하여 부드러움을 변경할 수 있습니다)
    # -----------------------------------
    
    frame_number = 0
    
    try:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            # ( ★★★ 2. 모든 프레임에 대해 카운터 증가 ★★★ )
            frame_number += 1

            # ( ★★★ 3. 10번째 프레임이 아니면 건너뛰기 ★★★ )
            # frame_number를 10으로 나눈 나머지가 0일 때만 아래 분석 로직을 실행합니다.
            if frame_number % 10 != 0:
                continue

            # --- [머리 방향 추정 로직] --- (10프레임마다 실행됨)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results_facemesh = face_mesh.process(rgb_frame)
            if results_facemesh.multi_face_landmarks:
                face_landmarks = results_facemesh.multi_face_landmarks[0]
                img_h, img_w, _ = frame.shape
                face_3d_model = np.array([[0.0, 0.0, 0.0], [0.0, -330.0, -65.0], [-225.0, 170.0, -135.0], [225.0, 170.0, -135.0], [-150.0, -150.0, -125.0], [150.0, -150.0, -125.0]], dtype=np.float64)
                landmark_idx = [1, 152, 263, 33, 291, 61]
                face_2d_points = np.array([ [face_landmarks.landmark[idx].x * img_w, face_landmarks.landmark[idx].y * img_h] for idx in landmark_idx ], dtype=np.float64)
                focal_length = img_w
                cam_matrix = np.array([[focal_length, 0, img_w / 2], [0, focal_length, img_h / 2], [0, 0, 1]])
                dist_coeffs = np.zeros((4, 1), dtype=np.float64)
                success, rot_vec, trans_vec = cv2.solvePnP(face_3d_model, face_2d_points, cam_matrix, dist_coeffs)
                rot_mat, _ = cv2.Rodrigues(rot_vec)
                sy = np.sqrt(rot_mat[0, 0] * rot_mat[0, 0] + rot_mat[1, 0] * rot_mat[1, 0])
                singular = sy < 1e-6
                if not singular:
                    x, y, _ = np.arctan2(rot_mat[2, 1], rot_mat[2, 2]), np.arctan2(-rot_mat[2, 0], sy), np.arctan2(rot_mat[1, 0], rot_mat[0, 0])
                else:
                    x, y, _ = np.arctan2(-rot_mat[1, 2], rot_mat[1, 1]), np.arctan2(-rot_mat[2, 0], sy), 0
                # 1. 원본(날것)의 각도 값 계산
                head_pitch = -np.degrees(x)
                head_yaw = -np.degrees(y)

                # 2. EMA 필터 적용
                if smooth_head_yaw is None: # 첫 프레임인 경우
                    smooth_head_yaw = head_yaw
                    smooth_head_pitch = head_pitch
                else:
                    smooth_head_yaw = alpha * head_yaw + (1 - alpha) * smooth_head_yaw
                    smooth_head_pitch = alpha * head_pitch + (1 - alpha) * smooth_head_pitch

                # 3. 부드러워진 값을 리스트에 추가
                head_pitches.append(smooth_head_pitch)
                head_yaws.append(smooth_head_yaw)

            # --- [시선 추정 로직] ---
            results_gaze = gaze_pipeline.step(frame)
            if results_gaze and results_gaze.pitch is not None and len(results_gaze.pitch) > 0:
                # 1. 원본(날것)의 각도 값 계산
                gaze_pitch = results_gaze.pitch[0]
                gaze_yaw = results_gaze.yaw[0]
                
                # 2. EMA 필터 적용
                if smooth_gaze_yaw is None: # 첫 프레임인 경우
                    smooth_gaze_yaw = gaze_yaw
                    smooth_gaze_pitch = gaze_pitch
                else:
                    smooth_gaze_yaw = alpha * gaze_yaw + (1 - alpha) * smooth_gaze_yaw
                    smooth_gaze_pitch = alpha * gaze_pitch + (1 - alpha) * smooth_gaze_pitch
                    
                # 3. 부드러워진 값을 리스트에 추가
                gaze_pitches.append(smooth_gaze_pitch)
                gaze_yaws.append(smooth_gaze_yaw)
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"비디오 분석 중 오류가 발생했습니다: {e}")
    finally:
        cap.release()
        os.unlink(temp_video_path)

    # --- [평균값 계산 및 결과 반환] --- (변경 없음)
    avg_head_yaw = sum(head_yaws) / len(head_yaws) if head_yaws else 0.0
    avg_head_pitch = sum(head_pitches) / len(head_pitches) if head_pitches else 0.0
    avg_gaze_yaw = sum(gaze_yaws) / len(gaze_yaws) if gaze_yaws else 0.0
    avg_gaze_pitch = sum(gaze_pitches) / len(gaze_pitches) if gaze_pitches else 0.0
    
    # Spring Boot의 CalibrationResultDto의 @JsonProperty에 맞춰 key 값을 변경합니다.
    response_content = {
        "head_yaw": float(avg_head_yaw),     # "avg_head_yaw" -> "head_yaw"
        "head_pitch": float(avg_head_pitch), # "avg_head_pitch" -> "head_pitch"
        "gaze_yaw": float(avg_gaze_yaw),     # "avg_gaze_yaw" -> "gaze_yaw"
        "gaze_pitch": float(avg_gaze_pitch)  # "avg_gaze_pitch" -> "gaze_pitch"
    }
    return JSONResponse(content=response_content)

# ==============================================================================
# 3. 면접 영상 분석 엔드포인트 (/analyze_video) - 새로 추가된 기능
# ==============================================================================
@app.post("/analyze_video")
async def analyze_interview_video(video_file: UploadFile = File(...),
                                  calib_hy: float = Form(...),
                                  calib_hp: float = Form(...),
                                  calib_gy: float = Form(...),
                                  calib_gp: float = Form(...)):
    """
    업로드된 면접 동영상을 10프레임 단위로 분석하고,
    EMA 필터를 적용한 시계열 데이터를 JSON 배열로 반환합니다.
    """
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video:
            contents = await video_file.read()
            temp_video.write(contents)
            temp_video_path = temp_video.name
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"임시 파일 생성에 실패했습니다: {e}")

    cap = cv2.VideoCapture(temp_video_path)
    if not cap.isOpened():
        os.unlink(temp_video_path)
        raise HTTPException(status_code=400, detail="업로드된 비디오 파일을 열 수 없거나 손상되었습니다.")

    # --- 변수 초기화 ---
    time_series_data = []
    frame_number = 0
    # EMA 필터를 위한 변수
    smooth_head_yaw, smooth_head_pitch = None, None
    smooth_gaze_yaw, smooth_gaze_pitch = None, None
    alpha = 0.2  # 스무딩 강도

    try:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            
            frame_number += 1
            if frame_number % 10 != 0:
                continue

            # --- AI 분석 로직 (머리 + 시선) ---
            head_yaw, head_pitch, gaze_yaw, gaze_pitch = None, None, None, None

            # [머리 방향 추정]
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results_facemesh = face_mesh.process(rgb_frame)
            if results_facemesh.multi_face_landmarks:
                face_landmarks = results_facemesh.multi_face_landmarks[0]
                # ... (solvePnP 및 각도 계산 로직은 이전과 동일)
                img_h, img_w, _ = frame.shape
                face_3d_model = np.array([[0.0, 0.0, 0.0], [0.0, -330.0, -65.0], [-225.0, 170.0, -135.0], [225.0, 170.0, -135.0], [-150.0, -150.0, -125.0], [150.0, -150.0, -125.0]], dtype=np.float64)
                landmark_idx = [1, 152, 263, 33, 291, 61]
                face_2d_points = np.array([ [face_landmarks.landmark[idx].x * img_w, face_landmarks.landmark[idx].y * img_h] for idx in landmark_idx ], dtype=np.float64)
                focal_length = img_w
                cam_matrix = np.array([[focal_length, 0, img_w / 2], [0, focal_length, img_h / 2], [0, 0, 1]])
                dist_coeffs = np.zeros((4, 1), dtype=np.float64)
                success, rot_vec, trans_vec = cv2.solvePnP(face_3d_model, face_2d_points, cam_matrix, dist_coeffs)
                rot_mat, _ = cv2.Rodrigues(rot_vec)
                sy = np.sqrt(rot_mat[0, 0] * rot_mat[0, 0] + rot_mat[1, 0] * rot_mat[1, 0])
                singular = sy < 1e-6
                if not singular:
                    x, y, _ = np.arctan2(rot_mat[2, 1], rot_mat[2, 2]), np.arctan2(-rot_mat[2, 0], sy), np.arctan2(rot_mat[1, 0], rot_mat[0, 0])
                else:
                    x, y, _ = np.arctan2(-rot_mat[1, 2], rot_mat[1, 1]), np.arctan2(-rot_mat[2, 0], sy), 0
                
                head_pitch = -np.degrees(x)
                head_yaw = -np.degrees(y)

            # [시선 추정]
            results_gaze = gaze_pipeline.step(frame)
            if results_gaze and results_gaze.pitch is not None and len(results_gaze.pitch) > 0:
                gaze_pitch = results_gaze.pitch[0]
                gaze_yaw = results_gaze.yaw[0]
            
            # --- EMA 필터 적용 및 데이터 저장 ---
            # 분석에 성공한 경우에만 데이터 처리
            if head_yaw is not None and gaze_yaw is not None:
                if smooth_head_yaw is None: # 첫 분석 프레임인 경우
                    smooth_head_yaw, smooth_head_pitch = head_yaw, head_pitch
                    smooth_gaze_yaw, smooth_gaze_pitch = gaze_yaw, gaze_pitch
                else:
                    smooth_head_yaw = alpha * head_yaw + (1 - alpha) * smooth_head_yaw
                    smooth_head_pitch = alpha * head_pitch + (1 - alpha) * smooth_head_pitch
                    smooth_gaze_yaw = alpha * gaze_yaw + (1 - alpha) * smooth_gaze_yaw
                    smooth_gaze_pitch = alpha * gaze_pitch + (1 - alpha) * smooth_gaze_pitch
                
                frame_data = {
                    "frame": frame_number,
                    "head_yaw": float(smooth_head_yaw),
                    "head_pitch": float(smooth_head_pitch),
                    "gaze_yaw": float(smooth_gaze_yaw),
                    "gaze_pitch": float(smooth_gaze_pitch)
                }
                time_series_data.append(frame_data)
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"비디오 분석 중 오류가 발생했습니다: {e}")
    finally:
        cap.release()
        os.unlink(temp_video_path)

    scores = calc_score(time_series_data, calib_hy, calib_hp, calib_gy, calib_gp)
    for frame_data, score in zip(time_series_data, scores):
        frame_data["score"] = score

    # 평균 점수 계산
    average_score = float(np.mean(scores)) if scores else 0.0

    # 최종 반환: 타임시리즈 데이터와 평균 점수를 함께 반환
    return JSONResponse(content={
        "time_series": time_series_data,
        "average_score": round(average_score)
    })

def calc_score(time_series, calib_hy, calib_hp, calib_gy, calib_gp):
    scores = []
    N = 9
    K = 0.1
    head_yaw_list, head_pitch_list, gaze_yaw_list, gaze_pitch_list = [], [], [], []
    for idx, row in enumerate(time_series):
        hy, hp, gy, gp = row["head_yaw"], row["head_pitch"], row["gaze_yaw"], row["gaze_pitch"]
        head_yaw_list.append(hy)
        head_pitch_list.append(hp)
        gaze_yaw_list.append(gy)
        gaze_pitch_list.append(gp)
        # 초기 9개는 누적 평균, 이후부터는 이동 평균
        if idx < N:
            mean_hy = np.mean(head_yaw_list)
            mean_hp = np.mean(head_pitch_list)
            mean_gy = np.mean(gaze_yaw_list)
            mean_gp = np.mean(gaze_pitch_list)
        else:
            mean_hy = np.mean(head_yaw_list[-N:])
            mean_hp = np.mean(head_pitch_list[-N:])
            mean_gy = np.mean(gaze_yaw_list[-N:])
            mean_gp = np.mean(gaze_pitch_list[-N:])
        abs_delta_hy = abs(hy - calib_hy)
        abs_delta_hp = abs(hp - calib_hp)
        abs_delta_gy = abs(gy - calib_gy)
        abs_delta_gp = abs(gp - calib_gp)
        rel_delta_hy = abs(hy - mean_hy)
        rel_delta_hp = abs(hp - mean_hp)
        rel_delta_gy = abs(gy - mean_gy)
        rel_delta_gp = abs(gp - mean_gp)
        w1, w2, w3, w4 = 0.25, 0.25, 0.25, 0.25
        S = (
            w1 * abs_delta_hy + w2 * abs_delta_hp + w3 * abs_delta_gy + w4 * abs_delta_gp +
            w1 * rel_delta_hy + w2 * rel_delta_hp + w3 * rel_delta_gy + w4 * rel_delta_gp
        )
        score = int(100 * math.exp(-K * S))
        scores.append(score)
    return scores

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=5003)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import torch
import os
import tempfile
import math
import random
from typing import Dict
import nest_asyncio
nest_asyncio.apply()
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from fastapi import Form

from l2cs import Pipeline

# --- 시드 고정 코드 ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
print("Seed fixed to 42")

print("FastAPI 서버를 시작합니다. AI 모델을 로딩합니다...")
app = FastAPI(title="AI Interview Analysis Server")
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5
)
device = "cuda" if torch.cuda.is_available() else "cpu"
try:
    gaze_pipeline = Pipeline(
        weights='models/L2CSNet_gaze360.pkl', arch='ResNet50', device=torch.device(device)
    )
    print(f"AI 모델 로딩 완료. Device: {device}")
except Exception as e:
    print(f"Error loading L2CS-Net model: {e}")

@app.post("/calibrate", response_model=Dict[str, float])
async def analyze_video(video_file: UploadFile = File(...)):
    print("Received /calibrate request")
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video:
            print("Creating temporary file for video")
            contents = await video_file.read()
            temp_video.write(contents)
            temp_video_path = temp_video.name
            print(f"Temporary video file saved at: {temp_video_path}")
    except Exception as e:
        print(f"Error creating temp file: {e}")
        raise HTTPException(status_code=500, detail=f"임시 파일 생성에 실패했습니다: {e}")

    head_yaws, head_pitches, gaze_yaws, gaze_pitches = [], [], [], []
    cap = cv2.VideoCapture(temp_video_path)
    if not cap.isOpened():
        os.unlink(temp_video_path)
        print("Failed to open video file")
        raise HTTPException(status_code=400, detail="업로드된 비디오 파일을 열 수 없거나 손상되었습니다.")

    smooth_head_yaw, smooth_head_pitch = None, None
    smooth_gaze_yaw, smooth_gaze_pitch = None, None
    alpha = 0.2
    frame_number = 0

    try:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Video frame read complete or failed")
                break

            frame_number += 1
            if frame_number % 10 != 0:
                continue

            print(f"Processing frame number: {frame_number}")

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results_facemesh = face_mesh.process(rgb_frame)
            if results_facemesh.multi_face_landmarks:
                print("Face detected")
                face_landmarks = results_facemesh.multi_face_landmarks[0]
                img_h, img_w, _ = frame.shape

                face_3d_model = np.array([[0.0, 0.0, 0.0], [0.0, -330.0, -65.0], [-225.0, 170.0, -135.0],
                                         [225.0, 170.0, -135.0], [-150.0, -150.0, -125.0], [150.0, -150.0, -125.0]], dtype=np.float64)
                landmark_idx = [1, 152, 263, 33, 291, 61]
                face_2d_points = np.array(
                    [[face_landmarks.landmark[idx].x * img_w, face_landmarks.landmark[idx].y * img_h] for idx in landmark_idx],
                    dtype=np.float64)
                focal_length = img_w
                cam_matrix = np.array([[focal_length, 0, img_w / 2], [0, focal_length, img_h / 2], [0, 0, 1]])
                dist_coeffs = np.zeros((4, 1), dtype=np.float64)
                success_pose, rot_vec, trans_vec = cv2.solvePnP(face_3d_model, face_2d_points, cam_matrix, dist_coeffs)
                if not success_pose:
                    print("solvePnP failed")
                    continue
                rot_mat, _ = cv2.Rodrigues(rot_vec)
                sy = np.sqrt(rot_mat[0, 0] ** 2 + rot_mat[1, 0] ** 2)
                singular = sy < 1e-6
                if not singular:
                    x, y, _ = np.arctan2(rot_mat[2, 1], rot_mat[2, 2]), np.arctan2(-rot_mat[2, 0], sy), np.arctan2(rot_mat[1, 0], rot_mat[0, 0])
                else:
                    x, y, _ = np.arctan2(-rot_mat[1, 2], rot_mat[1, 1]), np.arctan2(-rot_mat[2, 0], sy), 0

                head_pitch = -np.degrees(x)
                head_yaw = -np.degrees(y)
                print(f"Raw head_pitch: {head_pitch:.2f}, head_yaw: {head_yaw:.2f}")

                if smooth_head_yaw is None:
                    smooth_head_yaw = head_yaw
                    smooth_head_pitch = head_pitch
                else:
                    smooth_head_yaw = alpha * head_yaw + (1 - alpha) * smooth_head_yaw
                    smooth_head_pitch = alpha * head_pitch + (1 - alpha) * smooth_head_pitch
                print(f"Smoothed head_pitch: {smooth_head_pitch:.2f}, head_yaw: {smooth_head_yaw:.2f}")

                head_pitches.append(smooth_head_pitch)
                head_yaws.append(smooth_head_yaw)
            else:
                print(f"No face detected in frame {frame_number}")

            try:
                results_gaze = gaze_pipeline.step(frame)
                if results_gaze and results_gaze.pitch is not None and len(results_gaze.pitch) > 0:
                    gaze_pitch = results_gaze.pitch[0]
                    gaze_yaw = results_gaze.yaw[0]
                    print(f"Raw gaze_pitch: {gaze_pitch:.2f}, gaze_yaw: {gaze_yaw:.2f}")

                    if smooth_gaze_yaw is None:
                        smooth_gaze_yaw = gaze_yaw
                        smooth_gaze_pitch = gaze_pitch
                    else:
                        smooth_gaze_yaw = alpha * gaze_yaw + (1 - alpha) * smooth_gaze_yaw
                        smooth_gaze_pitch = alpha * gaze_pitch + (1 - alpha) * smooth_gaze_pitch
                    print(f"Smoothed gaze_pitch: {smooth_gaze_pitch:.2f}, gaze_yaw: {smooth_gaze_yaw:.2f}")

                    gaze_pitches.append(smooth_gaze_pitch)
                    gaze_yaws.append(smooth_gaze_yaw)
                else:
                    print(f"Gaze data invalid or empty for frame {frame_number}")
            except Exception as e:
                print(f"Error during gaze pipeline processing: {e}")
                raise HTTPException(status_code=500, detail=f"Gaze 분석 중 오류가 발생했습니다: {e}")

    except Exception as e:
        print(f"Error during video analysis: {e}")
        raise HTTPException(status_code=500, detail=f"비디오 분석 중 오류가 발생했습니다: {e}")
    finally:
        cap.release()
        print("VideoCapture released")
        try:
            os.unlink(temp_video_path)
            print(f"Temporary file deleted: {temp_video_path}")
        except Exception as e:
            print(f"Error deleting temporary file: {e}")

    avg_head_yaw = sum(head_yaws) / len(head_yaws) if head_yaws else 0.0
    avg_head_pitch = sum(head_pitches) / len(head_pitches) if head_pitches else 0.0
    avg_gaze_yaw = sum(gaze_yaws) / len(gaze_yaws) if gaze_yaws else 0.0
    avg_gaze_pitch = sum(gaze_pitches) / len(gaze_pitches) if gaze_pitches else 0.0

    response_content = {
        "head_yaw": float(avg_head_yaw),
        "head_pitch": float(avg_head_pitch),
        "gaze_yaw": float(avg_gaze_yaw),
        "gaze_pitch": float(avg_gaze_pitch)
    }
    print(f"Returning response: {response_content}")
    return JSONResponse(content=response_content)

@app.post("/analyze_video")
async def analyze_interview_video(video_file: UploadFile = File(...),
                                  calib_hy: float = Form(...),
                                  calib_hp: float = Form(...),
                                  calib_gy: float = Form(...),
                                  calib_gp: float = Form(...)):
    print("Received /analyze_video request")
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video:
            print("Creating temporary file for analysis video")
            contents = await video_file.read()
            temp_video.write(contents)
            temp_video_path = temp_video.name
            print(f"Temporary video file path: {temp_video_path}")
    except Exception as e:
        print(f"Error creating temporary file: {e}")
        raise HTTPException(status_code=500, detail=f"임시 파일 생성에 실패했습니다: {e}")

    cap = cv2.VideoCapture(temp_video_path)
    if not cap.isOpened():
        os.unlink(temp_video_path)
        print("Cannot open uploaded video file or file is corrupted")
        raise HTTPException(status_code=400, detail="업로드된 비디오 파일을 열 수 없거나 손상되었습니다.")

    time_series_data = []
    frame_number = 0
    smooth_head_yaw, smooth_head_pitch = None, None
    smooth_gaze_yaw, smooth_gaze_pitch = None, None
    alpha = 0.2

    try:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("End of video reached or read error")
                break

            frame_number += 1
            if frame_number % 10 != 0:
                continue

            print(f"Processing frame #{frame_number}")

            head_yaw, head_pitch, gaze_yaw, gaze_pitch = None, None, None, None
            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results_facemesh = face_mesh.process(rgb_frame)
            if results_facemesh.multi_face_landmarks:
                face_landmarks = results_facemesh.multi_face_landmarks[0]
                img_h, img_w, _ = frame.shape
                face_3d_model = np.array([[0.0, 0.0, 0.0],
                                         [0.0, -330.0, -65.0],
                                         [-225.0, 170.0, -135.0],
                                         [225.0, 170.0, -135.0],
                                         [-150.0, -150.0, -125.0],
                                         [150.0, -150.0, -125.0]], dtype=np.float64)
                landmark_idx = [1, 152, 263, 33, 291, 61]
                face_2d_points = np.array(
                    [[face_landmarks.landmark[idx].x * img_w, face_landmarks.landmark[idx].y * img_h] for idx in landmark_idx],
                    dtype=np.float64)
                focal_length = img_w
                cam_matrix = np.array([[focal_length, 0, img_w / 2],
                                       [0, focal_length, img_h / 2],
                                       [0, 0, 1]])
                dist_coeffs = np.zeros((4, 1), dtype=np.float64)
                success_pose, rot_vec, trans_vec = cv2.solvePnP(face_3d_model, face_2d_points, cam_matrix, dist_coeffs)
                if not success_pose:
                    print(f"solvePnP failed in frame {frame_number}")
                else:
                    rot_mat, _ = cv2.Rodrigues(rot_vec)
                    sy = np.sqrt(rot_mat[0, 0] ** 2 + rot_mat[1, 0] ** 2)
                    singular = sy < 1e-6
                    if not singular:
                        x, y, _ = np.arctan2(rot_mat[2, 1], rot_mat[2, 2]), np.arctan2(-rot_mat[2, 0], sy), np.arctan2(rot_mat[1, 0], rot_mat[0, 0])
                    else:
                        x, y, _ = np.arctan2(-rot_mat[1, 2], rot_mat[1, 1]), np.arctan2(-rot_mat[2, 0], sy), 0
                    head_pitch = -np.degrees(x)
                    head_yaw = -np.degrees(y)
                    print(f"Frame {frame_number} raw head_pitch: {head_pitch:.2f}, head_yaw: {head_yaw:.2f}")

            results_gaze = gaze_pipeline.step(frame)
            if results_gaze and results_gaze.pitch is not None and len(results_gaze.pitch) > 0:
                gaze_pitch = results_gaze.pitch[0]
                gaze_yaw = results_gaze.yaw[0]
                print(f"Frame {frame_number} raw gaze_pitch: {gaze_pitch:.2f}, gaze_yaw: {gaze_yaw:.2f}")

            if head_yaw is not None and gaze_yaw is not None:
                if smooth_head_yaw is None:
                    smooth_head_yaw, smooth_head_pitch = head_yaw, head_pitch
                    smooth_gaze_yaw, smooth_gaze_pitch = gaze_yaw, gaze_pitch
                else:
                    smooth_head_yaw = alpha * head_yaw + (1 - alpha) * smooth_head_yaw
                    smooth_head_pitch = alpha * head_pitch + (1 - alpha) * smooth_head_pitch
                    smooth_gaze_yaw = alpha * gaze_yaw + (1 - alpha) * smooth_gaze_yaw
                    smooth_gaze_pitch = alpha * gaze_pitch + (1 - alpha) * smooth_gaze_pitch

                frame_data = {
                    "frame": frame_number,
                    "head_yaw": float(smooth_head_yaw),
                    "head_pitch": float(smooth_head_pitch),
                    "gaze_yaw": float(smooth_gaze_yaw),
                    "gaze_pitch": float(smooth_gaze_pitch)
                }
                print(f"Frame {frame_number} smoothed data: {frame_data}")
                time_series_data.append(frame_data)

    except Exception as e:
        print(f"Error during video analysis: {e}")
        raise HTTPException(status_code=500, detail=f"비디오 분석 중 오류가 발생했습니다: {e}")
    finally:
        cap.release()
        print("VideoCapture released")
        try:
            os.unlink(temp_video_path)
            print(f"Temporary file deleted: {temp_video_path}")
        except Exception as e:
            print(f"Error deleting temporary file: {e}")

    scores = calc_score(time_series_data, calib_hy, calib_hp, calib_gy, calib_gp)
    for frame_data, score in zip(time_series_data, scores):
        frame_data["score"] = score
        print(f"Frame {frame_data['frame']} score: {score}")

    average_score = float(np.mean(scores)) if scores else 0.0
    print(f"Average score: {average_score}")

    return JSONResponse(content={
        "time_series": time_series_data,
        "average_score": round(average_score)
    })

def calc_score(time_series, calib_hy, calib_hp, calib_gy, calib_gp):
    scores = []
    N = 9
    K = 0.1
    head_yaw_list, head_pitch_list, gaze_yaw_list, gaze_pitch_list = [], [], [], []
    for idx, row in enumerate(time_series):
        hy, hp, gy, gp = row["head_yaw"], row["head_pitch"], row["gaze_yaw"], row["gaze_pitch"]
        head_yaw_list.append(hy)
        head_pitch_list.append(hp)
        gaze_yaw_list.append(gy)
        gaze_pitch_list.append(gp)
        if idx < N:
            mean_hy = np.mean(head_yaw_list)
            mean_hp = np.mean(head_pitch_list)
            mean_gy = np.mean(gaze_yaw_list)
            mean_gp = np.mean(gaze_pitch_list)
        else:
            mean_hy = np.mean(head_yaw_list[-N:])
            mean_hp = np.mean(head_pitch_list[-N:])
            mean_gy = np.mean(gaze_yaw_list[-N:])
            mean_gp = np.mean(gaze_pitch_list[-N:])
        abs_delta_hy = abs(hy - calib_hy)
        abs_delta_hp = abs(hp - calib_hp)
        abs_delta_gy = abs(gy - calib_gy)
        abs_delta_gp = abs(gp - calib_gp)
        rel_delta_hy = abs(hy - mean_hy)
        rel_delta_hp = abs(hp - mean_hp)
        rel_delta_gy = abs(gy - mean_gy)
        rel_delta_gp = abs(gp - mean_gp)
        w1, w2, w3, w4 = 0.25, 0.25, 0.25, 0.25
        S = (
            w1 * abs_delta_hy + w2 * abs_delta_hp + w3 * abs_delta_gy + w4 * abs_delta_gp +
            w1 * rel_delta_hy + w2 * rel_delta_hp + w3 * rel_delta_gy + w4 * rel_delta_gp
        )
        score = int(100 * math.exp(-K * S))
        scores.append(score)
    return scores

if __name__ == "__main__":
    import uvicorn
    print("Starting uvicorn server on 0.0.0.0:5003")
    uvicorn.run(app, host="0.0.0.0", port=5003)


Seed fixed to 42
FastAPI 서버를 시작합니다. AI 모델을 로딩합니다...
AI 모델 로딩 완료. Device: cuda
Starting uvicorn server on 0.0.0.0:5003


INFO:     Started server process [16028]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5003 (Press CTRL+C to quit)


Received /calibrate request
Creating temporary file for video
Temporary video file saved at: C:\Users\User\AppData\Local\Temp\tmp9cd6gf4_.mp4
Processing frame number: 10
Face detected
Raw head_pitch: -0.16, head_yaw: 25.73
Smoothed head_pitch: -0.16, head_yaw: 25.73
Raw gaze_pitch: -0.29, gaze_yaw: 0.28
Smoothed gaze_pitch: -0.29, gaze_yaw: 0.28
Processing frame number: 20
Face detected
Raw head_pitch: -3.43, head_yaw: 33.80
Smoothed head_pitch: -0.82, head_yaw: 27.34
Raw gaze_pitch: -0.03, gaze_yaw: 0.23
Smoothed gaze_pitch: -0.23, gaze_yaw: 0.27
Processing frame number: 30
Face detected
Raw head_pitch: -3.79, head_yaw: 33.25
Smoothed head_pitch: -1.41, head_yaw: 28.52
Raw gaze_pitch: -0.44, gaze_yaw: 0.37
Smoothed gaze_pitch: -0.28, gaze_yaw: 0.29
Video frame read complete or failed
VideoCapture released
Temporary file deleted: C:\Users\User\AppData\Local\Temp\tmp9cd6gf4_.mp4
Returning response: {'head_yaw': 27.19860636313094, 'head_pitch': -0.7968247062894834, 'gaze_yaw': 0.27984469